In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.ticker as mticker
import time
import csv
import dateutil.parser, dateutil.tz as tz
import zoneinfo
from sklearn.preprocessing import MultiLabelBinarizer
import json

In [7]:
with open('../data/google_reviews/meta-Alaska.json', 'r') as f:
    data = [pd.json_normalize(json.loads(line)) for line in f]

# Concatenate the list of dataframes into a single dataframe
df = pd.concat(data, ignore_index=True)
display(df)

JSONDecodeError: Expecting value: line 2 column 1 (char 766)

In [8]:
rating_fn = '../data/google_reviews/rating-Vermont.csv'
meta_fn = '../data/google_reviews/meta-Vermont.json'
# headers = ["user_id","parent_asin","rating","timestamp"]
rating_df = pd.read_csv(rating_fn, header=0)
display(rating_df)
with open(meta_fn, 'r') as f:
    data = [pd.json_normalize(json.loads(line)) for line in f]

meta_df = pd.concat(data, ignore_index=True)
# meta_df = pd.read_json(meta_fn)
# display(meta_df)

,business,user,rating,timestamp
0,0x89e02445cb9db457:0x37f42bff4edf7a43,104490523034805396984,5,1629987079369
1,0x89e02445cb9db457:0x37f42bff4edf7a43,112062684820633635547,5,1626442303453
2,0x89e02445cb9db457:0x37f42bff4edf7a43,110048311042721875910,5,1627319568225
3,0x89e02445cb9db457:0x37f42bff4edf7a43,106174372090234957142,5,1625223629975
4,0x89e02445cb9db457:0x37f42bff4edf7a43,106238734329148428463,5,1630520930973
...,...,...,...,...
841905,0x89e1c79c60aef325:0xbed8bfc41018f020,101031262113246567699,5,1503368927511
841906,0x89e1c79c60aef325:0xbed8bfc41018f020,109751562687532742649,5,1508992170442
841907,0x89e1c79c60aef325:0xbed8bfc41018f020,113511800890575185422,5,1566750149101
841908,0x89e1c79c60aef325:0xbed8bfc41018f020,107697312145259650604,5,1521208726615


JSONDecodeError: Expecting value: line 2 column 1 (char 1101)

In [4]:
df = pd.merge(rating_df, meta_df, left_on=["business"], right_on=["gmap_id"])
df = df[["timestamp", "category"]]
df.dropna(inplace=True)
categories = df["category"].to_numpy()
unique_categories = list(set([item for sublist in categories for item in sublist]))
# for category in unique_categories:
#     df[category] = df['category'].apply(lambda x: 1 if category in x else 0)
# df.drop('category', axis=1, inplace=True)
display(df)

,timestamp,category
0,1629987079369,"[Security system supplier, Fire protection equ..."
1,1629987079369,"[Security system supplier, Fire protection equ..."
2,1626442303453,"[Security system supplier, Fire protection equ..."
3,1626442303453,"[Security system supplier, Fire protection equ..."
4,1627319568225,"[Security system supplier, Fire protection equ..."
...,...,...
843251,1503368927511,"[Orchard, Farm]"
843252,1508992170442,"[Orchard, Farm]"
843253,1566750149101,"[Orchard, Farm]"
843254,1521208726615,"[Orchard, Farm]"


In [5]:
mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('category')),
                index=df.index,
                columns=mlb.classes_))
display(df)

# np_df = df.to_numpy()

,timestamp,3D printing service,ATM,ATV dealer,ATV rental service,ATV repair shop,Accountant,Accounting firm,Accounting school,Acupuncture clinic,...,Woodworking supply store,Work clothes store,Yamaha motorcycle dealer,Yarn store,Yoga retreat center,Yoga studio,Youth center,Youth hostel,Youth organization,Youth social services organization
0,1629987079369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1629987079369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1626442303453,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1626442303453,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1627319568225,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843251,1503368927511,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843252,1508992170442,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843253,1566750149101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843254,1521208726615,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def timestamp_to_timeseries(timestamps, start_time, end_time, bin_size):
    hist, _ = np.histogram(timestamps, bins=np.arange(start_time, end_time+bin_size, bin_size))
    return hist

def groupby(df, group_header='category_id', timestamp_header='timestamp'):
    np_df = df[[group_header, timestamp_header]].to_numpy()
    groups, groups_index = np.unique(np_df[:, 0], return_index=True)
    return np.split(np_df[:,1], groups_index[1:]), groups

def timeseries_to_csv(timeseries, group, csv_fn):
    timeseries_group = [group]
    timeseries_group.extend(list(timeseries))
    with open(csv_fn, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(timeseries_group)

In [6]:
# df['time']=pd.to_datetime(df['timestamp'],unit='s')+datetime.timedelta(hours=8)
start_time = datetime.datetime(2007, 1, 1, 00, 00).timestamp()
end_time = datetime.datetime(2021, 9, 30, 00, 00).timestamp()
bin_size = 60*60 # in seconds
# csv_fn = '../data/google_reviews_vermont.csv'
timestamp_header = 'timestamp'
group_header = 'category'
print(start_time, end_time)

# timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
# timestamps_grouped, groups= groupby(df, group_header, timestamp_header)
# for timestamps, group in zip(timestamps_grouped, groups):
#     timeseries = timestamp_to_timeseries(timestamps, start_time, end_time, bin_size)
#     timeseries_to_csv(timeseries, group, csv_fn)

# 1602133857705

1167638400.0 1632985200.0


In [7]:
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District_of_Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina", "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", "Wisconsin", "Wyoming"]
# states = ["West_Virginia", "Wisconsin", "Wyoming"]
for state in states:
    start_count_time = time.time()
    rating_fn = f'../data/google_reviews/rating-{state}.csv'
    meta_fn = f'../data/google_reviews/meta-{state}.json'
    csv_fn = f'../data/google_reviews/timeseries_{state}_hourly.csv'
    # headers = ["user_id","parent_asin","rating","timestamp"]
    rating_df = pd.read_csv(rating_fn, header=0)
    meta_df = pd.read_json(meta_fn)
    print('data load time', time.time() - start_count_time)
    df = pd.merge(rating_df, meta_df, left_on=["business"], right_on=["gmap_id"])
    df = df[["timestamp", "category"]]
    df.dropna(inplace=True)

    mlb = MultiLabelBinarizer(sparse_output=True)

    df = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop('category')),
                    index=df.index,
                    columns=mlb.classes_))

    timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
    for i, category in enumerate(df.columns[1:]):
        np_df = df[['timestamp', category]].to_numpy()
        category_np = np_df[np_df[:,1]==1, 0] // 1000
        hist = timestamp_to_timeseries(category_np, start_time, end_time, bin_size)
        timeseries_to_csv(hist, category, csv_fn)
    print(f'{state} took {time.time()-start_count_time} seconds')

data load time 6.032813549041748
Alabama took 206.34989500045776 seconds
data load time 0.8592395782470703
Alaska took 42.130271673202515 seconds
data load time 12.190363645553589
Arizona took 451.4396562576294 seconds
data load time 4.1651999950408936
Arkansas took 138.7792603969574 seconds
data load time 56.130860805511475
California took 2199.1487169265747 seconds
data load time 25.987911462783813
Colorado took 417.832483291626 seconds
data load time 4.521362781524658
Connecticut took 142.27690649032593 seconds
data load time 1.377190351486206
Delaware took 60.63455772399902 seconds
data load time 1.4578564167022705
District_of_Columbia took 50.29712986946106 seconds
data load time 44.61304306983948
Florida took 1679.765031337738 seconds
data load time 21.824496030807495
Georgia took 623.9021716117859 seconds
data load time 2.9802300930023193
Hawaii took 86.55922150611877 seconds
data load time 3.138667106628418
Idaho took 114.89557528495789 seconds
data load time 19.73058319091797


In [8]:
timeseries_fn = f'../data/google_reviews_hourly.csv'
timeseries_df = pd.read_csv(timeseries_fn, header=0)
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District_of_Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina", "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", "Wisconsin", "Wyoming"]
for state in states:
    state_csv = f'../data/google_reviews/timeseries_{state}_hourly.csv'
    timeseries_state_df = pd.read_csv(state_csv, header=0)
    timeseries_df = pd.concat([timeseries_df, timeseries_state_df], axis=0)
    timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
timeseries_df.to_csv(timeseries_fn, index=False)

C:\Users\Coen\AppData\Local\Temp\ipykernel_81092\3115059659.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
C:\Users\Coen\AppData\Local\Temp\ipykernel_81092\3115059659.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
C:\Users\Coen\AppData\Local\Temp\ipykernel_81092\3115059659.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

In [20]:
timeseries_fn = f'../data/google_reviews_weekly.csv'
timeseries_df = pd.read_csv(timeseries_fn, header=0)
display(timeseries_df)

,category,1167638400.0,1168243200.0,1168848000.0,1169452800.0,1170057600.0,1170662400.0,1171267200.0,1171872000.0,1172476800.0,...,1627286400.0,1627891200.0,1628496000.0,1629100800.0,1629705600.0,1630310400.0,1630915200.0,1631520000.0,1632124800.0,1632729600.0
3D printing service,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,1,0,0,0,0,0
ATM,0,0,0,0,0,0,0,0,0,0,...,949,646,537,296,146,19,0,0,0,0
ATV dealer,0,0,0,0,0,0,0,0,0,0,...,42,26,21,15,18,0,0,0,0,0
ATV rental service,0,0,0,0,0,0,0,0,0,0,...,40,21,18,8,4,0,0,0,0,0
ATV repair shop,0,0,0,0,0,0,0,0,0,0,...,61,42,25,10,15,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Youth organization,0,0,0,0,0,0,0,0,0,0,...,52,37,27,13,11,0,0,0,0,0
Youth social services organization,0,0,0,0,0,0,0,0,0,0,...,11,2,2,0,0,0,0,0,0,0
Yucatan restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zoo,0,0,0,0,0,0,0,0,0,0,...,34,29,8,3,5,0,0,0,0,0


In [26]:
start_date = datetime.datetime(2020, 1, 1, 00, 00).timestamp()
print(start_date)

1577865600.0


In [20]:
cols = list(timeseries_df.columns)
for i in range(1000):
    if start_date+200*i in cols:
        print(start_date+200*i)
        break
# .index(start_date+200*3)

In [30]:
print(cols[1000*100*2])

1550930400.0


In [42]:
print(timeseries_df.iloc[:,1000*100*2:])

     1550930400.0 1550934000.0 1550937600.0 1550941200.0 1550944800.0  \
0               0            0            0            0            0   
1              73           63           81           79           75   
2              13            9           14           18           32   
3               0            1            2            2            1   
4               7            4            5            7           16   
...           ...          ...          ...          ...          ...   
4564            2            2            2            9            1   
4565            0            0            0            1            1   
4566            0            0            0            0            0   
4567           22           22           30           26           31   
4568            0            0            0            0            0   

     1550948400.0 1550952000.0 1550955600.0 1550959200.0 1550962800.0  ...  \
0               0            0            0  

In [44]:
timeseries_df_cut = pd.concat([timeseries_df.iloc[:,0], timeseries_df.iloc[:,1000*100*2:]], axis=1)

In [45]:
print(timeseries_df_cut)

                                category 1550930400.0 1550934000.0  \
0                    3D printing service            0            0   
1                                    ATM           73           63   
2                             ATV dealer           13            9   
3                     ATV rental service            0            1   
4                        ATV repair shop            7            4   
...                                  ...          ...          ...   
4564                  Youth organization            2            2   
4565  Youth social services organization            0            0   
4566                  Yucatan restaurant            0            0   
4567                                 Zoo           22           22   
4568                           shipwreck            0            0   

     1550937600.0 1550941200.0 1550944800.0 1550948400.0 1550952000.0  \
0               0            0            0            0            0   
1            

In [47]:
timeseries_fn = f'../data/google_reviews_hourly_cut.csv'

timeseries_df_cut.to_csv(timeseries_fn, index=False)

In [ ]:
index = np.sum(timeseries_df_cut.iloc[:, 1:], axis=1) > 1000000

In [67]:
ind = np.nonzero(index)

In [69]:
timeseries_df_cut_top = timeseries_df_cut.iloc[ind]

In [72]:
timeseries_fn = f'../data/google_reviews_hourly_cut_top.csv'
timeseries_df_cut_top.to_csv(timeseries_fn, index=False)